In [1]:
import pandas as pd
import numpy as np
import folium
import simplejson,requests
import sys
import time
import import_ipynb
import YouTubeAPI as youtube

importing Jupyter notebook from YouTubeAPI.ipynb
                    Date                                              Title  \
0   2022-05-13T05:11:45Z  용인 에버랜드 갈때 100% 즐기기 위한 꿀팁 ✨ 자유이용권, 주차, 머리띠, 스마...   
1   2022-05-09T01:07:53Z  충북 진천여행🌿  대전 근교 하루 여행 코스 가볼만한곳 BEST  한반도 지형 전망...   
2   2022-04-27T04:49:28Z  경주 뚜벅이 여행코스 BEST10✨ 경주 여행 브이로그, 황리단길 카페와 맛집, 대...   
3   2022-04-09T10:47:00Z  공모전 수상작🎡 국내여행 베스트 대구 근교 가볼만한곳 ✨ 경북 주말 여행지 / 국내...   
4   2022-03-30T04:18:14Z  구례 여행 코스 가볼만한곳 BEST3🌿 지리산 치즈랜드, 산수유마을, 화엄사 홍매화...   
5   2022-03-21T06:30:55Z  죽기전에 꼭 가봐야할 국내여행 베스트10 ✨ 나만 알고싶은 여행지/ 인스타 핫플/ ...   
6   2022-03-03T11:08:31Z  등산 브이로그 등린이 vlog #11 충북 단양팔경 월악산 제비봉  등산로, 등산코...   
7   2022-02-22T07:31:20Z  국내 힐링여행지 전남 여행 보성 가볼만한곳 BEST (w. 다향아트밸리/율포해수욕장...   
8   2022-02-18T12:50:12Z        국내여행지 추천#12 충청도 여행 베스트10 (충남 당일치기 사진찍기좋은 곳)   
9   2022-02-11T07:39:44Z  경기도 당일치기로 가볼만한곳 제부도 여행코스 BEST10 ✨ 서울근교 가볼만한곳/ ...   
10  2022-01-25T05:26:39Z  국내 액티비티 끝판왕! 충청도 공주 이색 데이트 공주 경비행기 체험 여행 브이로그 ...   
11 

In [2]:
# 장소 추출 예시를 위한 코드
lex_desc = youtube.ex_desc.split('\n')  # 각 줄마다 split
spl_place = []
for a in lex_desc:                      # 타임라인에 존재하는 장소 추출
    if a.find(":") == 2:
        spl_place.append(a[6:])
    else:
        continue

place = []
for i in spl_place:
    if i == "Intro":                    # Intro는 제거
        continue
    elif "/" in i:
        place.append(i.split(" / ")[0]) # /로 구분된 장소 split
        place.append(i.split(" / ")[1])
    elif "#" in i:
        place.append(i.split(" #")[1])  # # 뒤에 존재하는 장소 추출
    elif "와 "in i:
        place.append(i.split("와 ")[0]) # 와 뒤에 존재하는 장소 추출
        place.append(i.split("와 ")[1])
    else:
        place.append(i)
print(place)


['영양 자작나무숲 ', '영주무섬다리', '예천 뿅뿅다리', '포항 곤륜산', '예천 회룡포 전망대', '포항 내연산 소금강 전망대', '포항 스페이스워크', '청송 송소고택', '포항 이가리 닻 전망대', '포항 호미반도', '울릉해담길', '예천 활체험센터', '삼강문화단지', '문경 관광사격장과 에코랄라', '포항 장기읍성', '청도 청도읍성', '문경 고모산성']


In [3]:
place_id = []
place_do =  []
place_si =[]
place_name = []
place_address = []
place_phone = []
place_url = []
place_X = []
place_Y = []
kakao_df = pd.DataFrame(columns=['ID','이름','주소','도', '시',
                                 '전화번호','URL','X좌표','Y좌표'])

url = "https://dapi.kakao.com/v2/local/search/keyword.json?" # kakaomap api에 접근
apikey = "c533ba443ba909af8d277567163d03fe"
for i in range(len(place)):    
    query = str(place[i])
    r = requests.get( url, params = {'query':query}, 
                     headers={'Authorization' : 'KakaoAK ' + apikey } )
    js = simplejson.JSONEncoder().encode(r.json())
    
    # 장소를 입력하여 json파일 형태의 데이터 생성
    rj = r.json()                       
    
    # 입력했을 때 결과가 없으면 생략
    if rj['documents'] == []:                        
        continue
    place_id.append(rj['documents'][0]['id'])         # 이후 list에 결과 append
    if rj['meta']['same_name']['selected_region'] == '':
        place_do.append(rj['documents'][0]['address_name'].split()[0])
        place_si.append(rj['documents'][0]['address_name'].split()[1])
    else:
        place_do.append(rj['meta']['same_name']['selected_region'].split()[0])
        place_si.append(rj['meta']['same_name']['selected_region'].split()[1])
    place_name.append(rj['documents'][0]['place_name'])
    place_address.append(rj['documents'][0]['address_name'])
    place_phone.append(rj['documents'][0]['phone'])
    place_url.append(rj['documents'][0]['place_url'])
    place_X.append(rj['documents'][0]['x'])
    place_Y.append(rj['documents'][0]['y'])

In [4]:
kakao_df=pd.DataFrame([place_id,place_name,place_address,place_do,place_si,place_phone,place_url,place_X,place_Y]).T
kakao_df.columns=['placeID','pname','paddress',
                  'pdo', 'psi','pphone','pdaumURL','pX','pY']
kakao_df                                              # 데이터프레임 생성

,placeID,pname,paddress,pdo,psi,pphone,pdaumURL,pX,pY
0,1726199741,영양 자작나무숲,경북 영양군 수비면 죽파리 산 39-1,경북,영양군,,http://place.map.kakao.com/1726199741,129.26464356321975,36.742549456062
1,11623349,외나무다리,경북 영주시 문수면 탄산리 766,경북,영주시,,http://place.map.kakao.com/11623349,128.61942389375,36.7300208222057
2,20263100,뿅뿅다리쉼터,경북 예천군 용궁면 대은리 349,경북,예천군,,http://place.map.kakao.com/20263100,128.327065979419,36.5794683277883
3,25178704,곤륜산,경북 포항시 북구 흥해읍 칠포리 산 85-1,경북,포항시,,http://place.map.kakao.com/25178704,129.39158115005458,36.13469443760141
4,20035376,회룡대,경북 예천군 지보면 마산리 산 113-1,경북,예천군,054-650-6395,http://place.map.kakao.com/20035376,128.315197354288,36.5782050189438
5,1440340624,환호공원 스페이스워크,경북 포항시 북구 두호동 산 8,경북,포항시,054-270-8282,http://place.map.kakao.com/1440340624,129.390266451304,36.0650661624599
6,27149596,송소고택,경북 청송군 파천면 덕천리 176,경북,청송군,054-874-6556,http://place.map.kakao.com/27149596,129.02640558673312,36.436239232964596
7,446522573,이가리 닻전망대,경북 포항시 북구 청하면 이가리 산 67-3,경북,포항시,054-270-3204,http://place.map.kakao.com/446522573,129.379108982736,36.1878660301882
8,213160097,호미반도해안둘레길,경북 포항시 남구 동해면 입암리 350,경북,포항시,,http://place.map.kakao.com/213160097,129.481897009713,36.0155580430111
9,36690600,활기찬 예천활체험센터,경북 예천군 예천읍 청복리 467-1,경북,예천군,054-653-2434,http://place.map.kakao.com/36690600,128.46674078386096,36.636595448945954
